# Lab 6 

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import io
import linearmodels as lm


from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS


In [2]:
df = pd.read_csv("panel-for-R.csv")
df.head(5)

,idnum,panelwave,ballot,form,formwt,oversamp,sample,panstat_2,panstat_3,mar1,...,wtpan12,wtpan123,wtpannr12,wtpannr123,xmarsex,xmovie,xnorcsiz,year,yearval,zodiac
0,9,1,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,2.0,1.0,2006.0,NaN,9.0
1,9,2,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2008.0,NaN,9.0
2,9,3,3,2,1,1,9,1,1,1.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2010.0,NaN,9.0
3,10,1,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2006.0,NaN,3.0
4,10,2,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2008.0,NaN,3.0


### realrinc: Family income on 1972-2006 surveys in constant dollars
### relpersn: To what extent do you consider yourself a religious person, from 1) very religious to 4)not religious at all
### hrs1: Last week were you working full time, part time, going to school, keeping house, or what? A. IF WORKING, FULL OR PART TIME:  How many hours did you work last week, at all jobs?

### In the regression model, I want to explore what factors can affect family income. In general, working hours and personal age are more likely considered to affect family income, that's why I have add it here as the independent variable. Also, I feel it's interesting to see whether being religious will affect the family income. Therefore, the regression will explain the relationship between independent variables, hours worked, age, and religious, and dependent variable family income. 

# 1. Run a naive ("pooled") OLS regression on the panel data. Tell we how you expect your Xs to affect your Y and why.   Interpret your results.


In [3]:
lm_happy=smf.ols(formula = 'realrinc ~ age + hrs1 + relpersn', data=df).fit()
lm_happy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               realrinc   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     43.41
Date:                Fri, 08 Dec 2023   Prob (F-statistic):           2.65e-27
Time:                        16:49:37   Log-Likelihood:                -28899.
No. Observations:                2365   AIC:                         5.781e+04
Df Residuals:                    2361   BIC:                         5.783e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -3.36e+04   5851.107     -5.743      0.000   -4.51e+04   -2.21e+04
age          528.9951     80.774      6.549      0.000     370.600     687.391
hrs1         671.2517     71.798      9.349      0.000     530.459     812.044
relpersn    4228.0702   1088.015      3.886      0.000    2094.506    6361.634
==============================================================================
Omnibus:                     3123.209   Durbin-Watson:                   1.556
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           425860.733
Skew:                           7.521   Prob(JB):                         0.00
Kurtosis:                      66.995   Cond. No.                         362.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### R-square = 0.052, it means 5.2% of  family income is explained by the age, hours worked, and being religious in the simple OLS. There were a positive relationship between hour worked, age, and religious, and family income. Net of the hour worked and being religious, it appears that for with every year that get older, on average, about 529 dollars more in family income is earned. It's statistically significant and it explains about 5.2 percent of the variances in salary earned. Net of the age and being religious, it appears that for every hour more worked, on average, about 671 dollars more in family income is earned. It's statistically significant and it explains about 5.2 percent of the variances in salary earned. Net of the age and hours worked, it appears that for people being one unit less religious, on average, about 4228 dollars less in family income is earned. It's statistically significant and it explains about 5.2 percent of the variances in salary earned. Overall, we can see age, hours worked, and being religious are statistically significant with pvalue closed to 0. We could say they posistive affect the family income. 

In [ ]:
# before run the log -log model, recode log variables

In [4]:
df['lnage'] = np.log(df['age'])
df['lnrealrinc'] = np.log(df['realrinc'])
df['lnhrs1'] = np.log(df['hrs1'])
df['lnrelpersn'] = np.log(df['relpersn'])

In [5]:
df[['lnage','lnrealrinc','lnhrs1','lnrelpersn']].describe()

,lnage,lnrealrinc,lnhrs1,lnrelpersn
count,4761.000000,2847.000000,2802.000000,4768.000000
mean,3.825847,9.592979,3.649706,0.757447
std,0.367444,1.162260,0.479530,0.448370
min,2.890372,5.556828,0.000000,0.000000
25%,3.555348,9.131094,3.637586,0.693147
50%,3.871201,9.824241,3.688879,0.693147
75%,4.110874,10.290534,3.912023,1.098612
max,4.488636,13.081842,4.488636,1.386294


In [6]:
lm_happy2=smf.ols(formula = 'lnrealrinc ~ lnage + lnhrs1 + lnrelpersn', data=df).fit()
lm_happy2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             lnrealrinc   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     173.4
Date:                Fri, 08 Dec 2023   Prob (F-statistic):          1.38e-101
Time:                        16:49:44   Log-Likelihood:                -3226.9
No. Observations:                2365   AIC:                             6462.
Df Residuals:                    2361   BIC:                             6485.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8022      0.307     12.403      0.000       3.201       4.403
lnage          0.6760      0.065     10.413      0.000       0.549       0.803
lnhrs1         0.8833      0.042     20.902      0.000       0.800       0.966
lnrelpersn     0.2051      0.046      4.423      0.000       0.114       0.296
==============================================================================
Omnibus:                      342.519   Durbin-Watson:                   1.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1678.527
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       6.950   Cond. No.                         86.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### this model i run a same model as an elasticity, log-log model. R^2 increased compared with first OLS model that about 18.1% variance in family income is explained by age, working hours, and religious personality, so now explaining almost 18.1% of the variance in log salary. This is also quite statistically significant. For every 1% increase in hours worked, on average, we can see a 0.833% increase in the family income, net of age and relpersn. For every 1% increase in being unreligious, on averge, we can see a 0.2051% increase in family income, net of age and working hours. For every 1% increase in age of being elder, on average, we can see a 0.676% increase in family income, net of working hours and relpersn. 

# 2. Run a first differences regression on the same model in Question 1. Interpret your results. Do you draw a different conclusion than in Question 1? Explain.




In [7]:
df = df.set_index(["idnum", "panelwave"])

In [9]:
exog_vars1 = ['age','hrs1','relpersn'] # multiple variables 
exog1 = df[exog_vars1]
lm_fd = FirstDifferenceOLS(df.realrinc, exog1)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
res_fd

/Users/yemei/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2311: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,realrinc,R-squared:,0.0016
Estimator:,FirstDifferenceOLS,R-squared (Between):,0.2822
No. Observations:,1077,R-squared (Within):,0.0006
Date:,"Fri, Dec 08 2023",R-squared (Overall):,0.2393
Time:,16:53:10,Log-likelihood,-1.353e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,0.5561
Entities:,1221,P-value,0.6441
Avg Obs:,1.9369,Distribution:,"F(3,1074)"
Min Obs:,1.0000,,
Max Obs:,3.0000,F-statistic (robust):,0.9225


### By regressing changes in working hours, age, and religious personality to change in family income, we can see that there appears to be no association between changes in age, hrs1, and changes in realrinc given the high p value that means statistically insignificant. Also for relpersn, it has a low pvalue (0.11) that suggesting it might be a significant predictor for the variable realrinc. Overall, this model explains a small proportion about 0.16% of the overall variance in family income. 

In [10]:
exog_vars2 = ['age','hrs1','relpersn']
exog2 = df[exog_vars2]
lm_fd2 = FirstDifferenceOLS(df.lnrealrinc, exog2)
res_fd2 = lm_fd2.fit(cov_type='clustered', cluster_entity=True)
res_fd2

/Users/yemei/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2311: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,lnrealrinc,R-squared:,0.0103
Estimator:,FirstDifferenceOLS,R-squared (Between):,0.2031
No. Observations:,1077,R-squared (Within):,0.0201
Date:,"Fri, Dec 08 2023",R-squared (Overall):,0.2027
Time:,16:54:13,Log-likelihood,-1293.3
Cov. Estimator:,Clustered,,
,,F-statistic:,3.7377
Entities:,1221,P-value,0.0109
Avg Obs:,1.9369,Distribution:,"F(3,1074)"
Min Obs:,1.0000,,
Max Obs:,3.0000,F-statistic (robust):,3.7087


### Now, the model improved that the relationship between changes in log working hours and age, and log family income are now marginally statistically significant as p-value is not larger than 0.05. It is in the expected direction that for each 1% change in working hours, on average, there is a 0.49% change in family income, net of the age and relpersn. Also for each 1% change in age, on average, there isa 1.62% change in family income, net of the hrs1 and relpersn.  While for relpersn is not statistically significant at the 0.05 level, indicating that there may not be enough evidence to conclde a significant relationship between the being religious personality and the family income. R^2 shows that these changes explain about 1.03% in the variance in logged family income. 
### compared with all models in the previous, we can clearly seen that log-log first difference model ( log-differncing) performs better on explaining how age, hrs1, relpersn associated with realrinc(family income). This model give a more interpretable and meaningful on its result.